In [1]:
%load_ext autoreload
%autoreload 2
%cd ..

/Users/akkirr/Desktop/IT/annotated-diffusion


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from pathlib import Path

import torch
from torch import nan_to_num
from torchvision import transforms, models
from torch.utils.data import DataLoader

import numpy as np
from PIL import Image
import requests

from datasets import load_dataset
from torchvision.utils import save_image
from torch.optim import Adam

In [3]:
from mylib import *
from lora import *

In [4]:
model = models.resnet101()
# model

In [5]:
inject_lora(
    model,
    1,
    0.1,
    ['Bottleneck'],
    [nn.Conv2d],
    [LoraInjectedConv2d],
    verbose=True
)

Injected lora (64x1x64) in layer1.0.conv1
Injected lora (64x1x64) in layer1.0.conv2
Injected lora (64x1x256) in layer1.0.conv3
Injected lora (64x1x256) in layer1.0.0
Injected lora (256x1x64) in layer1.1.conv1
Injected lora (64x1x64) in layer1.1.conv2
Injected lora (64x1x256) in layer1.1.conv3
Injected lora (256x1x64) in layer1.2.conv1
Injected lora (64x1x64) in layer1.2.conv2
Injected lora (64x1x256) in layer1.2.conv3
Injected lora (256x1x128) in layer2.0.conv1
Injected lora (128x1x128) in layer2.0.conv2
Injected lora (128x1x512) in layer2.0.conv3
Injected lora (256x1x512) in layer2.0.0
Injected lora (512x1x128) in layer2.1.conv1
Injected lora (128x1x128) in layer2.1.conv2
Injected lora (128x1x512) in layer2.1.conv3
Injected lora (512x1x128) in layer2.2.conv1
Injected lora (128x1x128) in layer2.2.conv2
Injected lora (128x1x512) in layer2.2.conv3
Injected lora (512x1x128) in layer2.3.conv1
Injected lora (128x1x128) in layer2.3.conv2
Injected lora (128x1x512) in layer2.3.conv3
Injected l

In [8]:
inject_lora(
    model,
    1,
    0.1,
    ['Bottleneck'],
    [nn.Conv2d],
    verbose=True
)

In [14]:
import pytest
import torch
from torch import nn
from torch.optim import Adam
from copy import deepcopy

from lora import *
from lora.lora import LORA_MODULES
from lora.utils import isinstance_by_class

class Attention(nn.Module):
    def __init__(self):
        super().__init__()
        self.QKV = nn.Linear(1, 1)
        self.C = nn.Linear(1, 1)
        self.lrelu = nn.LeakyReLU()

    def forward(self, x):
        return self.C(self.lrelu(self.QKV(x)))


class TimeEmbedding(nn.Module):
    def __init__(self):
        super().__init__()
        self.time_proj = nn.Linear(1, 1)
        self.lrelu = nn.LeakyReLU()

    def forward(self, x):
        return self.lrelu(self.time_proj(x))

class A(nn.Module):
    def __init__(self):
        super().__init__()
        self.just_linear = nn.Linear(1, 1)
        self.attn = Attention()
        self.time_embedder = TimeEmbedding()

    def forward(self, x):
        return self.attn(self.just_linear(x) + self.time_embedder(x))

def models_eq(sd1, sd2, keys):
    return all(torch.allclose(sd1[key], sd2[key]) for key in keys)

def models_neq(sd1, sd2, keys):
    return any(not torch.allclose(sd1[key], sd2[key]) for key in keys)

def find_lora_modules(model: nn.Module, lora_modules=LORA_MODULES):
    for name, module in model.named_modules():
        if isinstance_by_class(module, lora_modules):
            yield name


def main():
    model = A()
    optim = Adam(model.parameters())

    inject_lora(
        model,
        2, 0,
        ["Attention"],
        [nn.Linear],
        [LoraInjectedLinear],
        verbose=True
    )
    sd1 = deepcopy(model.state_dict())


    x = torch.tensor([[1]], dtype=torch.float32)
    loss = model(x).mean()
    loss.backward()
    optim.step()
    optim.zero_grad()

    sd2 = deepcopy(model.state_dict())

    all_keys = set(sd2.keys())
    lora_keys = {}
    for module_name in find_lora_modules(model):
        print(list(model.get_submodule(module_name).named_parameters))
main()
    

Injected lora (1x2x1) in attn.QKV
Injected lora (1x2x1) in attn.C
Injected lora (1x2x1) in attn.src_linear
Injected lora (1x2x2) in attn.lora_down
Injected lora (2x2x1) in attn.lora_up
Injected lora (1x2x1) in attn.src_linear
Injected lora (1x2x2) in attn.lora_down
Injected lora (2x2x1) in attn.lora_up
Injected lora (1x2x1) in attn.src_linear
Injected lora (1x2x2) in attn.lora_down
Injected lora (2x2x1) in attn.lora_up
Injected lora (1x2x1) in attn.src_linear
Injected lora (1x2x2) in attn.lora_down
Injected lora (2x2x1) in attn.lora_up
Injected lora (1x2x1) in attn.src_linear
Injected lora (1x2x2) in attn.lora_down
Injected lora (2x2x1) in attn.lora_up
Injected lora (1x2x1) in attn.src_linear
Injected lora (1x2x2) in attn.lora_down
Injected lora (2x2x1) in attn.lora_up
Injected lora (1x2x1) in attn.src_linear
Injected lora (1x2x2) in attn.lora_down
Injected lora (2x2x1) in attn.lora_up
Injected lora (1x2x1) in attn.src_linear
Injected lora (1x2x2) in attn.lora_down
Injected lora (2x2x1

RecursionError: maximum recursion depth exceeded while calling a Python object